
###🧪 Camada Silver – Tratamento e Enriquecimento

In [0]:
# Importação
from pyspark.sql.functions import trim, col, when, lit
from pyspark.sql.types import StringType

# Leitura das tabelas bronze
df_transacoes = spark.table("adb_cliente_savana_prd.daniel_bronze.transacoes")
df_clientes = spark.table("adb_cliente_savana_prd.daniel_bronze.clientes")

# Renomear coluna ID para ID_Cliente (para join)
df_clientes = df_clientes.withColumnRenamed("ID", "ID_Cliente")

# Join
df_silver = df_transacoes.join(df_clientes, on="ID_Cliente", how="left")

# Limpeza de espaços extras
for campo in ["Nome", "Cidade", "Meio_de_Pgmt"]:
    if campo in df_silver.columns:
        df_silver = df_silver.withColumn(campo, trim(col(campo)))

# Correções manuais de erros de Nome conhecidos
df_silver = df_silver.withColumn("Nome",
    when(col("Nome") == "Ana", "Ana Costa")
    .when(col("Nome") == "Carlos", "Carlos Pereira")
    .when(col("Nome") == "Fernanda", "Fernanda Gomes")
    .when(col("Nome") == "Gabriel", "Gabriel Martins")
    .when(col("Nome") == "Jos", "José da Silva")
    .when(col("Nome") == "Jos da Silva", "José da Silva")
    .when(col("Nome") == "Maria", "Maria Oliveira")
    .when(col("Nome") == "Mariana", "Mariana Almeida")
    .when(col("Nome") == "Ricardo", "Ricardo Lima")
    .when(col("Nome") == "Roberto", "Roberto Santos")
    .when(col("Nome") == "Tatiane", "Tatiane Ramos")
    .otherwise(col("Nome"))
)

df_silver = df_silver.withColumn("Cidade",
    when(col("Cidade") == "So Paulo", "São Paulo")
    .when(col("Cidade") == "Bralia", "Brasília")
    .when(col("Cidade") == "Braslia", "Brasília")
    .when(col("Cidade") == "Goinia", "Goiânia")
    .when(col("Cidade") == "Joo Pessoa", "João Pessoa")
    .otherwise(col("Cidade"))
)

def padronizar_nulls(df):
    for campo in df.schema.names:
        if df.schema[campo].dataType == StringType():
            df = df.withColumn(campo, when(trim(col(campo)) == "", None).otherwise(col(campo)))
    return df

# Aplicar ao df_silver
df_silver = padronizar_nulls(df_silver)

#----------------------------------------------------------------------
# Salvar na camada Silver
#df_silver.write.format("delta").mode("overwrite") \
#    .saveAsTable("adb_cliente_savana_prd.daniel_silver.BaseClientes_Transacoes")

# Exibir
display(df_silver)